In [2]:
import BioSimSpace as BSS
import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from time import process_time

/home/rzhu/miniconda3/envs/BSS/lib/python3.7/site-packages/BioSimSpace/Parameters/_process.py:134: UserWarning: Attempting to parameterise a molecule without hydrogen atoms!
  _warnings.warn("Attempting to parameterise a molecule without hydrogen atoms!")


In [34]:
def getFreeEnergy(title, ligand, ref_ligand, protein, lam_bound, lam_free):
    
    # Parametrise the ligand
    ligand = BSS.Parameters.gaff2(ligand, work_dir = './lig_par/' + title).getMolecule()
    
    # Morph and combine systems
    mapping = BSS.Align.matchAtoms(ligand, ref_ligand)
    ligand = BSS.Align.rmsdAlign(ligand, ref_ligand, mapping)
    merged = BSS.Align.merge(ligand, ref_ligand, mapping)
    complx = merged + protein 

    # Solvate the protein ligand complex in a box of water molecules
    complx_sol = BSS.Solvent.tip3p(molecule = complx, box = 3*[90*BSS.Units.Length.angstrom])
    # Solvate the merged ligand itself in a smaller box of water molecules
    merged_sol = BSS.Solvent.tip3p(molecule = merged, box = 3*[40*BSS.Units.Length.angstrom])
    
    # Create the minimisation protocol
    prot_minimisation = BSS.Protocol.Minimisation(steps = 2000)
    # Minimise both legs
    proc_minimisation_complx = BSS.MD.run(complx_sol, 
                                          prot_minimisation, 
                                          work_dir = './minimisation/' + title + '/complex')
    complx_minimised = proc_minimisation_complx.getSystem(block = True)
    proc_minimisation_merged = BSS.MD.run(merged_sol, 
                                          prot_minimisation, 
                                          work_dir = './minimisation/' + title + '/merged')
    merged_minimised = proc_minimisation_merged.getSystem(block = True)

    # Create the equilibration protocol
    prot_equilibration = BSS.Protocol.Equilibration(timestep = 2*BSS.Units.femtosecond,
                                                    runtime = 2*BSS.Units.nanosecond)
    # Equilibrate both legs
    proc_equilibration_complx = BSS.MD.run(complx_minimised,
                                           prot_equilibration,
                                           work_dir = './equilibration/' + title + '/complex')
    complx_equilibrated = proc_equilibration_complx.getSystem(block = True)
    proc_equilibration_merged = BSS.MD.run(merged_minimised, 
                                           prot_equilibration,
                                           work_dir = './equilibration/' + title + '/merged')
    merged_equilibrated = proc_equilibrationn_merged.getSystem(block = True)

    # Create the free energy protocols
    prot_bound = BSS.Protocol.FreeEnergy(timestep = 2*BSS.Units.Time.femtosecond,
                                         runtime = 2*BSS.Units.nanosecond,
                                         num_lam = lam_bound)
    prot_free = BSS.Protocol.FreeEnergy(timestep = 2*BSS.Units.Time.femtosecond,
                                        runtime = 2*BSS.Units.nanosecond,
                                        num_lam = lam_free)
    # Initialise the free energy object for each leg
    fep_bound = BSS.FreeEnergy.Relative(complx_equilibrated,
                                        prot_bound,
                                        engine = "somd",
                                        work_dir = './somd/' + title + "/bound")
    fep_free = BSS.FreeEnergy.Relative(merged_equilibrated,
                                       prot_free,
                                       engine = "somd",
                                       work_dir = './somd/' + title + "/free")
    
    # Get the PMF and the overlap matrix
    pmf_bound, overlap_bound = fep_bound.analyse()
    pmf_free, overlap_free = fep_free.analyse()
    
    # Compute the relative free-energy difference with PMF
    binding_free_energy = BSS.FreeEnergy.Relative.difference(pmf_bound, pmf_free)
    
    return binding_free_energy, overlap_bound, overlap_free

In [32]:
# Plot and save heatmaps of overlap matrixes

def heatmap(bound, free, title):
    fig, axs = plt.subplots(ncols = 2, nrows = 1, figsize = (20, 10), constrained_layout = True)
    fig.supxlabel('λ index',fontsize = 14)
    fig.supylabel('λ index',fontsize = 14)
    
    cmap = mpl.cm.viridis
    bounds = [0, 0.1, 0.2, 0.4, 0.6, 0.8]
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    fig.colorbar(mpl.cm.ScalarMappable(norm = norm, cmap = cmap))
    
    for col in range(2):
        if col == 0:
            leg = "Bound leg"
            matrix = np.array(bound)
        else:
            leg = "Free leg"
            matrix = np.array(free)
        lamb = matrix.shape[0]

        im = axs[col].imshow(matrix, norm = norm, cmap = cmap)   
        
        axs[col].set_xticks(np.arange(lamb), labels = np.arange(lamb))
        axs[col].set_yticks(np.arange(lamb), labels = np.arange(lamb))
        
        axs[col].set_title(leg, fontsize=16)
        axs[col].spines[:].set_visible(False)
        axs[col].grid(visible = True, which = "minor", axis = 'both', color = "w", linestyle = '-', linewidth = 3)
        
        for i in range(lamb):
            for j in range(lamb):
                text = axs[col].text(j, i, '{:.2f}'.format(matrix[i, j]), ha = "center", va = "center", color = "w", fontsize = 12)
    
    fig.savefig('overlap_heatmaps/' + title)

In [ ]:
### Run FEPs and save the results

df = pd.DataFrame()
testset = pd.read_csv("testset.csv")
if not os.path.exists('overlap_heatmaps'): os.makedirs('overlap_heatmaps')
    
lam_bound = 20
lam_free = 12

# Set the first entry as the reference ligand
ref_ligand = BSS.Parameters.gaff2(testset['SMILES'][0], work_dir = "./reflig_par").getMolecule()
protein = BSS.Parameters.ff14SB(BSS.IO.readMolecules(['./protein_par/leap.crd','./protein_par/leap.top'])[0], work_dir = "./protein_par_1").getMolecule()

for index, row in testset[1:3].iterrows():
    title = row['Molecule Name']
    ligand = row['SMILES']
#    try:
    FreeEnergy, overlap_bound, overlap_free = getFreeEnergy(title, ligand, ref_ligand, protein, lam_bound, lam_free)
    df = df.append({'ligand': title, 'RelativeFE': str(FreeEnergy[0]), 'STD': (FreeEnergy[1])}, ignore_index = True)
    heatmap(overlap_bound, overlap_free, title)
#    except:
#        df = df.append({'ligand': title, 'Error': '1'}, ignore_index = True)

df.to_csv('FEP_result.csv', index = False)